In [1]:
from tool import *
def fill_albedo2(month,type,sza_L,interp_2000=False):
    max_distance=3
    distence_w_dim=torch.load(root_path+'information_val2/weights.pth')[type]
    sza_L_month=sza_L[0,month-1]
    # distence_w_dim=[1,1,1,1]
    bands=['albedo_BSA_vis', 
        'albedo_WSA_vis', 
        'albedo_BSA_nir', 
        'albedo_WSA_nir', 
        'albedo_BSA_shortwave', 
        'albedo_WSA_shortwave']
    # band='albedo_BSA_vis'

    # m_index=[(month-3)%12+1,(month-2)%12+1,(month-1)%12+1,(month)%12+1,(month+1)%12+1]
    m_index=[(month-2)%12+1,(month-1)%12+1,(month)%12+1]
    offset=[1 if m_index[j]-m_index[0]>=0 else -1 for j in range(len(m_index))]
    if np.array(offset).sum()==len(offset):
        offset=[0]*len(m_index)
    # print(month,m_index,offset)

    vs=[]
    ms=[]
    land_proportions=[]
    for i in range(len(offset)):    
        NC=nc.Dataset(root_path+f'information_temp/month_{m_index[i]}.nc','r')
        for band in bands:
            vs.append(NC.variables[f'{band}-{type}'][:])
        if type!='ocean':
            ms.append(NC.variables[f'area-{type}'][:])
        # qa=NC.variables['qa'][:]
        area_grid=NC.variables['area-grid'][:]
        area_land=NC.variables['area-land'][:]
        land_proportions.append(area_land/area_grid)
        NC.close()
    offset_bands=[]
    for i in offset:
        offset_bands+=[i]*len(bands)
    if offset[0]!=0:
        nan_array=np.zeros((1,180,360))*np.nan
        vs=[np.concatenate((nan_array,vs[i]),axis=0) if offset_bands[i]==1 else np.concatenate((vs[i],nan_array),axis=0) for i in range(len(offset_bands))]
        land_proportions=[np.concatenate((nan_array,land_proportions[i]),axis=0) if offset[i]==1 else np.concatenate((land_proportions[i],nan_array),axis=0) for i in range(len(offset))]
        if type!='ocean':
            ms=[np.concatenate((nan_array,ms[i]),axis=0) if offset[i]==1 else np.concatenate((ms[i],nan_array),axis=0) for i in range(len(offset))]
        else:
            ms=land_proportions  
    else:
        if type=='ocean':
            ms=land_proportions  
    M,H,W=vs[0].shape
    v=np.stack(vs).reshape((len(offset),len(bands),M,H,W)).transpose((1,0,2,3,4))#(6, 3, 21, 180, 360)

    # vv=[v[:,i] for i in range(len(bands))]
    m=np.stack(ms)
    # plt.imshow(v[0,1,1])
    # plt.show()
    # land_proportion=np.stack(land_proportions)
    # print(month,m_index,v.shape)
    all_indices = np.indices(v[0].shape).reshape(4, -1).T
    
    

    
    
    if interp_2000:
        if type!='ocean':
                interp_indices = all_indices[((v[0]==2000) & (m>0)).reshape(-1)]
        else:
            interp_indices = all_indices[((v[0]==2000) & (m<1) & (m>0)).reshape(-1)]
        nonan_indices=all_indices[((~np.isnan(v[0])).reshape(-1)) & ((v[0]!=2000).reshape(-1))]
    else:
    
        if type!='ocean':
            interp_indices = all_indices[(np.isnan(v[0]) & (m>0)).reshape(-1)]
        else:
            interp_indices = all_indices[(np.isnan(v[0]) & (m<1) & (m>0)).reshape(-1)]
        nonan_indices=all_indices[(~np.isnan(v[0])).reshape(-1)]
    
    
    
    
    # print(len(nonan_indices))
    interp_indices=[j for j in interp_indices if j[0]==int(len(offset)/2)]  
    interp_indices=[j for j in interp_indices if sza_L_month[j[2],j[3]]<2] #只插值天顶角小于85的
    nonan_indices2=[j for j in nonan_indices if j[0]==int(len(offset)/2)]  
    nonan_indices2=[j for j in nonan_indices2 if sza_L_month[j[2],j[3]]<2]
    # interp_values=[]
    # search_n=1
    # print(month,type,len(interp_indices),round(len(interp_indices)/(len(nonan_indices)+len(interp_indices))*100,2),'%')
    str_output=f'{month}-{type}  '+str(len(interp_indices))+'  '+str(round(len(interp_indices)/(len(nonan_indices2)+len(interp_indices)+0.01)*100,2))+'%'
    # for interp_indice in tqdm(interp_indices):
    #     k=TS_IDM_4d(interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)
    #     while np.isnan(k):
    #         search_n*=2
    #         k=TS_IDM_4d(interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)

    #     interp_values.append(k)
    time.sleep(1)
    kk=[search_nonan2(v,interp_indice,max_distance=max_distance,distence_w_dim=distence_w_dim,interp_2000=interp_2000) for interp_indice in tqdm(interp_indices,desc=str_output)]
    interp_values=np.array([a[0] for a in kk])
    dws=np.array([a[1] for a in kk])
    # print('fill value '+str((1-np.isnan(interp_values).sum()/interp_values.shape[0])*100)+'%')
    dws_v=np.zeros(v.shape)
    for z in range(len(interp_indices)):
        v[:,interp_indices[z][0],interp_indices[z][1],interp_indices[z][2],interp_indices[z][3]]=np.array(interp_values[z])
        dws_v[:,interp_indices[z][0],interp_indices[z][1],interp_indices[z][2],interp_indices[z][3]]=np.array(dws[z])
    # print(v.shape,dws_v.shape)
    if offset[int(len(offset)/2)]>=0:
        return v[:,int(len(offset)/2),offset[int(len(offset)/2)]:],dws_v[:,int(len(offset)/2),offset[int(len(offset)/2)]:]
    else:
        return v[:,int(len(offset)/2),:-1],dws_v[:,int(len(offset)/2),:-1]
def TS_IDM_4d_2(v,interp_indice,max_distance,distence_w_dim,interp_2000):
    
        value=v[zero(interp_indice[0]-int(max_distance/distence_w_dim[0])):interp_indice[0]+int(max_distance/distence_w_dim[0])+1,
                zero(interp_indice[1]-int(max_distance/distence_w_dim[1])):interp_indice[1]+int(max_distance/distence_w_dim[1])+1,
                zero(interp_indice[2]-int(max_distance/distence_w_dim[2])):interp_indice[2]+int(max_distance/distence_w_dim[2])+1,
                zero(interp_indice[3]-int(max_distance/distence_w_dim[3])):interp_indice[3]+int(max_distance/distence_w_dim[3])+1]
        non_nan_indices = np.where(~np.isnan(value))
        
        
        if interp_2000:
            v[v==2000]=np.nan
        
        
        non_nan_points = np.array(list(zip(non_nan_indices[0], non_nan_indices[1], non_nan_indices[2],non_nan_indices[3])))
         
        if len(non_nan_points)==0:
                interpolated_value=np.nan
                dws=np.nan
        else:
                k=np.array([zero(-interp_indice[0]+int(max_distance/distence_w_dim[0]))-int(max_distance/distence_w_dim[0]),
                        zero(-interp_indice[1]+int(max_distance/distence_w_dim[1]))-int(max_distance/distence_w_dim[1]),
                        zero(-interp_indice[2]+int(max_distance/distence_w_dim[2]))-int(max_distance/distence_w_dim[2]),
                        zero(-interp_indice[3]+int(max_distance/distence_w_dim[3]))-int(max_distance/distence_w_dim[3])])
                # print(k)
                # print(non_nan_points)
                non_nan_points+=k
                distance_s = np.array([np.linalg.norm(np.array(non_nan_points[i])*np.array(distence_w_dim)) for i in range(len(non_nan_points))])
                if len(distance_s)<10:
                        interpolated_value=np.nan
                        dws=np.nan
                else:
                        kth = np.partition(distance_s, 9)[9]
                        distance=distance_s.copy()
                        distance1=distance_s.copy()
                        distance1=distance1[distance_s<=kth]
                        weights = 1.0 / distance1**2
                        dws=np.sum(weights)
                        weights /= dws
                        non_nan_values = value[non_nan_indices]
                        non_nan_values=non_nan_values[distance<=kth]
                        interpolated_value = np.sum(weights * non_nan_values)
        return interpolated_value,dws
    
def search_nonan2(v,interp_indice,max_distance,distence_w_dim,interp_2000):
    k_bands=[]
    dws_bands=[]
    search_n=1
    for i in range(6):
        k,dws=TS_IDM_4d_2(v[i],interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim[i],interp_2000=interp_2000)
        while np.isnan(k):
            search_n*=2
            k,dws=TS_IDM_4d_2(v[i],interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim[i],interp_2000=interp_2000)
        k_bands.append(k)
        dws_bands.append(dws)
    
    return k_bands,dws_bands
def zero(k):
    if k<0:
        return 0
    else:
        return k

In [2]:
#fill albedo
bands=['albedo_BSA_vis', 
'albedo_WSA_vis', 
'albedo_BSA_nir', 
'albedo_WSA_nir', 
'albedo_BSA_shortwave',
'albedo_WSA_shortwave']
_,sza_L=torch.load(root_path+'sza_and_szaL.pth')
landtypes=['landtype'+str(i) for i in range(1,18)]+['snow']
for month in [10,11,12]:
    NC=nc.Dataset(root_path+f'information_val3/month_{month}.nc','r+')
    for type in landtypes:
        v_fill,dws=fill_albedo2(month,type,sza_L,True)
        for band_i in range(len(bands)):
            NC.variables[f'{bands[band_i]}-{type}'][:]=v_fill[band_i]
            try:
                NC.createVariable(f'DWS-{type}-{bands[band_i]}', 'f4', ("time", "latitude", "longitude"))
            except:
                pass
                # print(f'DWS-{type}-{bands[band_i]} have existed')
            NC.variables[f'DWS-{type}-{bands[band_i]}'][:]=dws[band_i]
    NC.close()

10-landtype14  1400  1.63%: 100%|██████████| 1400/1400 [00:24<00:00, 56.24it/s]
10-landtype15  0  0.0%: 0it [00:00, ?it/s]
11-landtype14  1096  1.3%: 100%|██████████| 1096/1096 [00:20<00:00, 52.51it/s]
11-landtype15  0  0.0%: 0it [00:00, ?it/s]
12-landtype14  862  1.14%: 100%|██████████| 862/862 [00:17<00:00, 49.51it/s]
12-landtype15  0  0.0%: 0it [00:00, ?it/s]
12-snow  8000  3.45%: 100%|██████████| 8000/8000 [02:29<00:00, 53.64it/s]


In [1]:
from tool import *
y=2001
m=1
fs=glob(root_path+f'snow/Snow/{str(y).zfill(4)+str(m).zfill(2)}*.nc')

In [2]:
f=fs[0]
nf=nc.Dataset(f)

In [9]:
type='vbd'
NC=nc.Dataset(root_path+f'white_sky_fraction/{type}sf.sfc.mon.mean.nc')

In [11]:
NC

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: COARDS
    description: Data is from NCEP initialized reanalysis
(4x/day).  It consists of T42  variables interpolated to
pressure surfaces from model (sigma) surfaces.
    platform: Model
    NCO: 20121013
    history: Created 2011/06/28 by ESRL/PSD Web & Data Team
Converted to chunked, deflated non-packed NetCDF4 2014/09
    title: monthly mean vbdsf.sfc from the NCEP Reanalysis
    dataset_title: NCEP-NCAR Reanalysis 1
    References: http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis.derived.html
    dimensions(sizes): lat(94), lon(192), time(909)
    variables(dimensions): float32 lat(lat), float32 lon(lon), float64 time(time), float32 vbdsf(time, lat, lon)
    groups: 

In [7]:
NC.variables[f'{type}sf']

<class 'netCDF4._netCDF4.Variable'>
float32 nbdsf(time, lat, lon)
    long_name: Monthly Mean Near IR Beam Downward Solar Flux at surface
    units: W/m^2
    precision: 1
    least_significant_digit: 0
    GRIB_id: 168
    GRIB_name: NBDSF
    var_desc: Near IR Beam Downward Solar Flux
    level_desc: Surface
    statistic: Mean
    parent_stat: Mean
    missing_value: -9.96921e+36
    valid_range: [-150.  700.]
    dataset: NCEP Reanalysis Derived Products
    actual_range: [ -7.1000977 285.3387   ]
unlimited dimensions: time
current shape = (907, 94, 192)
filling on, default _FillValue of 9.969209968386869e+36 used